In [1]:
import pandas as pd
import numpy as np

#plotting
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#model building
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


In [3]:
matches = pd.read_csv('/content/drive/MyDrive/IPL/IPL_Matches_2008_2022.csv')
deliveries = pd.read_csv('/content/drive/MyDrive/IPL/IPL_Ball_by_Ball_2008_2022.csv')
matches.head()

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,N,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,N,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,N,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,N,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan


In [4]:
deliveries.head()

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


In [5]:
matches.shape,deliveries.shape

((950, 20), (225954, 17))

In [6]:
matches.columns

Index(['ID', 'City', 'Date', 'Season', 'MatchNumber', 'Team1', 'Team2',
       'Venue', 'TossWinner', 'TossDecision', 'SuperOver', 'WinningTeam',
       'WonBy', 'Margin', 'method', 'Player_of_Match', 'Team1Players',
       'Team2Players', 'Umpire1', 'Umpire2'],
      dtype='object')

In [7]:
deliveries.columns

Index(['ID', 'innings', 'overs', 'ballnumber', 'batter', 'bowler',
       'non-striker', 'extra_type', 'batsman_run', 'extras_run', 'total_run',
       'non_boundary', 'isWicketDelivery', 'player_out', 'kind',
       'fielders_involved', 'BattingTeam'],
      dtype='object')

In [8]:
len(matches)

950

In [9]:
len(deliveries)

225954

In [10]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               950 non-null    int64  
 1   City             899 non-null    object 
 2   Date             950 non-null    object 
 3   Season           950 non-null    object 
 4   MatchNumber      950 non-null    object 
 5   Team1            950 non-null    object 
 6   Team2            950 non-null    object 
 7   Venue            950 non-null    object 
 8   TossWinner       950 non-null    object 
 9   TossDecision     950 non-null    object 
 10  SuperOver        946 non-null    object 
 11  WinningTeam      946 non-null    object 
 12  WonBy            950 non-null    object 
 13  Margin           932 non-null    float64
 14  method           19 non-null     object 
 15  Player_of_Match  946 non-null    object 
 16  Team1Players     950 non-null    object 
 17  Team2Players    

In [11]:
deliveries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225954 entries, 0 to 225953
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ID                 225954 non-null  int64 
 1   innings            225954 non-null  int64 
 2   overs              225954 non-null  int64 
 3   ballnumber         225954 non-null  int64 
 4   batter             225954 non-null  object
 5   bowler             225954 non-null  object
 6   non-striker        225954 non-null  object
 7   extra_type         12049 non-null   object
 8   batsman_run        225954 non-null  int64 
 9   extras_run         225954 non-null  int64 
 10  total_run          225954 non-null  int64 
 11  non_boundary       225954 non-null  int64 
 12  isWicketDelivery   225954 non-null  int64 
 13  player_out         11151 non-null   object
 14  kind               11151 non-null   object
 15  fielders_involved  7988 non-null    object
 16  BattingTeam        2

In [12]:
matches.isnull().sum()

ID                   0
City                51
Date                 0
Season               0
MatchNumber          0
Team1                0
Team2                0
Venue                0
TossWinner           0
TossDecision         0
SuperOver            4
WinningTeam          4
WonBy                0
Margin              18
method             931
Player_of_Match      4
Team1Players         0
Team2Players         0
Umpire1              0
Umpire2              0
dtype: int64

In [13]:
deliveries.isnull().sum()

ID                        0
innings                   0
overs                     0
ballnumber                0
batter                    0
bowler                    0
non-striker               0
extra_type           213905
batsman_run               0
extras_run                0
total_run                 0
non_boundary              0
isWicketDelivery          0
player_out           214803
kind                 214803
fielders_involved    217966
BattingTeam               0
dtype: int64

In [14]:
list1 = matches.columns.to_list()
remove_from_list = ['ID', 'Date', 'TossWinner', 'TossDecision', 'WinningTeam',
                    'WonBy', 'Player_of_Match', 'Venue',
                    'Umpire1', 'Umpire2']

for i in range(len(remove_from_list)):
    list1.remove(remove_from_list[i])


In [15]:
for i in list1:
    print('The unique values in', i, 'are: ', matches[i].unique())


The unique values in City are:  ['Ahmedabad' 'Kolkata' 'Mumbai' 'Navi Mumbai' 'Pune' 'Dubai' 'Sharjah'
 'Abu Dhabi' 'Delhi' 'Chennai' nan 'Hyderabad' 'Visakhapatnam'
 'Chandigarh' 'Bengaluru' 'Jaipur' 'Indore' 'Bangalore' 'Kanpur' 'Rajkot'
 'Raipur' 'Ranchi' 'Cuttack' 'Dharamsala' 'Kochi' 'Nagpur' 'Johannesburg'
 'Centurion' 'Durban' 'Bloemfontein' 'Port Elizabeth' 'Kimberley'
 'East London' 'Cape Town']
The unique values in Season are:  ['2022' '2021' '2020/21' '2019' '2018' '2017' '2016' '2015' '2014' '2013'
 '2012' '2011' '2009/10' '2009' '2007/08']
The unique values in MatchNumber are:  ['Final' 'Qualifier 2' 'Eliminator' 'Qualifier 1' '70' '69' '68' '67' '66'
 '65' '64' '63' '62' '61' '60' '59' '58' '57' '56' '55' '54' '53' '52'
 '51' '50' '49' '48' '47' '46' '45' '44' '43' '42' '41' '40' '39' '38'
 '37' '36' '35' '34' '33' '32' '31' '30' '29' '28' '27' '26' '25' '24'
 '23' '22' '21' '20' '19' '18' '17' '16' '15' '14' '13' '12' '11' '10' '9'
 '8' '7' '6' '5' '4' '3' '2' '1' 'Quali

In [16]:
list2 = deliveries.columns.to_list()
remove_from_list2 = ['ID', 'batter','innings', 'non-striker',
                     'bowler', 'fielders_involved', 'player_out']

for i in range(len(remove_from_list2)):
    list2.remove(remove_from_list2[i])

In [17]:
for i in list2:
    print('The unique values in', i, 'are: ', deliveries[i].unique())

The unique values in overs are:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
The unique values in ballnumber are:  [ 1  2  3  4  5  6  7  8  9 10]
The unique values in extra_type are:  [nan 'legbyes' 'wides' 'byes' 'noballs' 'penalty']
The unique values in batsman_run are:  [0 1 4 6 3 2 5]
The unique values in extras_run are:  [0 1 5 4 2 3 7]
The unique values in total_run are:  [0 1 4 6 3 2 5 7]
The unique values in non_boundary are:  [0 1]
The unique values in isWicketDelivery are:  [0 1]
The unique values in kind are:  [nan 'caught' 'caught and bowled' 'run out' 'bowled' 'stumped' 'lbw'
 'hit wicket' 'retired hurt' 'retired out' 'obstructing the field']
The unique values in BattingTeam are:  ['Rajasthan Royals' 'Gujarat Titans' 'Royal Challengers Bangalore'
 'Lucknow Super Giants' 'Sunrisers Hyderabad' 'Punjab Kings'
 'Delhi Capitals' 'Mumbai Indians' 'Chennai Super Kings'
 'Kolkata Knight Riders' 'Kings XI Punjab' 'Delhi Daredevils'
 'Rising Pune Supergiant' 'Guja

In [18]:
totalrun_df=deliveries.groupby(['ID','innings']).sum()['total_run'].reset_index()

totalrun_df

<ipython-input-18-daae2730f9f0>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  totalrun_df=deliveries.groupby(['ID','innings']).sum()['total_run'].reset_index()


,ID,innings,total_run
0,335982,1,222
1,335982,2,82
2,335983,1,240
3,335983,2,207
4,335984,1,129
...,...,...,...
1923,1312198,2,193
1924,1312199,1,157
1925,1312199,2,161
1926,1312200,1,130


In [19]:
totalrun_df = totalrun_df[totalrun_df['innings']==1]
totalrun_df['total_run'] = totalrun_df['total_run'].apply(lambda x:x+1)#to get target
totalrun_df

<ipython-input-19-16a02a375476>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalrun_df['total_run'] = totalrun_df['total_run'].apply(lambda x:x+1)#to get target


,ID,innings,total_run
0,335982,1,223
2,335983,1,241
4,335984,1,130
6,335985,1,166
8,335986,1,111
...,...,...,...
1918,1304116,1,158
1920,1312197,1,189
1922,1312198,1,208
1924,1312199,1,158


In [20]:
match_df = matches.merge(totalrun_df[['ID','total_run']],
                       left_on='ID',right_on='ID')

match_df

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,...,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,total_run
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,...,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,131
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,...,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon,158
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,...,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough,208
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,...,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma,189
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,...,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan,158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,335986,Kolkata,2008-04-20,2007/08,4,Kolkata Knight Riders,Deccan Chargers,Eden Gardens,Deccan Chargers,bat,...,Kolkata Knight Riders,Wickets,5.0,NaN,DJ Hussey,"['WP Saha', 'BB McCullum', 'RT Ponting', 'SC G...","['AC Gilchrist', 'Y Venugopal Rao', 'VVS Laxma...",BF Bowden,K Hariharan,111
946,335985,Mumbai,2008-04-20,2007/08,5,Mumbai Indians,Royal Challengers Bangalore,Wankhede Stadium,Mumbai Indians,bat,...,Royal Challengers Bangalore,Wickets,5.0,NaN,MV Boucher,"['L Ronchi', 'ST Jayasuriya', 'DJ Thornely', '...","['S Chanderpaul', 'R Dravid', 'LRPL Taylor', '...",SJ Davis,DJ Harper,166
947,335984,Delhi,2008-04-19,2007/08,3,Delhi Daredevils,Rajasthan Royals,Feroz Shah Kotla,Rajasthan Royals,bat,...,Delhi Daredevils,Wickets,9.0,NaN,MF Maharoof,"['G Gambhir', 'V Sehwag', 'S Dhawan', 'MK Tiwa...","['T Kohli', 'YK Pathan', 'SR Watson', 'M Kaif'...",Aleem Dar,GA Pratapkumar,130
948,335983,Chandigarh,2008-04-19,2007/08,2,Kings XI Punjab,Chennai Super Kings,"Punjab Cricket Association Stadium, Mohali",Chennai Super Kings,bat,...,Chennai Super Kings,Runs,33.0,NaN,MEK Hussey,"['K Goel', 'JR Hopes', 'KC Sangakkara', 'Yuvra...","['PA Patel', 'ML Hayden', 'MEK Hussey', 'MS Dh...",MR Benson,SL Shastri,241


In [21]:
match_df['Team1'].unique()

array(['Rajasthan Royals', 'Royal Challengers Bangalore',
       'Sunrisers Hyderabad', 'Delhi Capitals', 'Chennai Super Kings',
       'Gujarat Titans', 'Lucknow Super Giants', 'Kolkata Knight Riders',
       'Punjab Kings', 'Mumbai Indians', 'Kings XI Punjab',
       'Delhi Daredevils', 'Rising Pune Supergiant', 'Gujarat Lions',
       'Rising Pune Supergiants', 'Pune Warriors', 'Deccan Chargers',
       'Kochi Tuskers Kerala'], dtype=object)

In [22]:
teams = [
    'Sunrisers Hyderabad',
    'Mumbai Indians',
    'Royal Challengers Bangalore',
    'Kolkata Knight Riders',
    'Kings XI Punjab',
    'Chennai Super Kings',
    'Rajasthan Royals',
    'Delhi Capitals',
    'Lucknow Super Giants',
    'Gujarat Titans'
]

In [23]:
match_df = match_df[match_df['Team1'].isin(teams)]
match_df = match_df[match_df['Team2'].isin(teams)]

match_df['Team1'].unique()

array(['Rajasthan Royals', 'Royal Challengers Bangalore',
       'Delhi Capitals', 'Chennai Super Kings', 'Gujarat Titans',
       'Lucknow Super Giants', 'Sunrisers Hyderabad',
       'Kolkata Knight Riders', 'Mumbai Indians', 'Kings XI Punjab'],
      dtype=object)

In [24]:


# considering the match_id, city, winner, and total runs

match_df = match_df[['ID','City','WinningTeam','total_run']]

match_df

,ID,City,WinningTeam,total_run
0,1312200,Ahmedabad,Gujarat Titans,131
1,1312199,Ahmedabad,Rajasthan Royals,158
2,1312198,Kolkata,Royal Challengers Bangalore,208
3,1312197,Kolkata,Gujarat Titans,189
5,1304115,Mumbai,Mumbai Indians,160
...,...,...,...,...
942,335989,Chennai,Chennai Super Kings,209
944,335987,Jaipur,Rajasthan Royals,167
946,335985,Mumbai,Royal Challengers Bangalore,166
948,335983,Chandigarh,Chennai Super Kings,241


In [25]:
delivery_df = match_df.merge(deliveries,on='ID')

delivery_df.head(5)

,ID,City,WinningTeam,total_run_x,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,Ahmedabad,Gujarat Titans,131,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,Ahmedabad,Gujarat Titans,131,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,Ahmedabad,Gujarat Titans,131,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,Ahmedabad,Gujarat Titans,131,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,Ahmedabad,Gujarat Titans,131,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


In [26]:
delivery_df.columns

Index(['ID', 'City', 'WinningTeam', 'total_run_x', 'innings', 'overs',
       'ballnumber', 'batter', 'bowler', 'non-striker', 'extra_type',
       'batsman_run', 'extras_run', 'total_run_y', 'non_boundary',
       'isWicketDelivery', 'player_out', 'kind', 'fielders_involved',
       'BattingTeam'],
      dtype='object')

In [27]:
delivery_df.shape

(144108, 20)

In [28]:
delivery_df['current_score'] = delivery_df.groupby('ID').cumsum()['total_run_y']

delivery_df.head()

<ipython-input-28-1081ee16966f>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.cumsum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  delivery_df['current_score'] = delivery_df.groupby('ID').cumsum()['total_run_y']


,ID,City,WinningTeam,total_run_x,innings,overs,ballnumber,batter,bowler,non-striker,...,batsman_run,extras_run,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,current_score
0,1312200,Ahmedabad,Gujarat Titans,131,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,...,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,0
1,1312200,Ahmedabad,Gujarat Titans,131,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,...,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals,1
2,1312200,Ahmedabad,Gujarat Titans,131,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,...,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals,2
3,1312200,Ahmedabad,Gujarat Titans,131,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,...,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,2
4,1312200,Ahmedabad,Gujarat Titans,131,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,...,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,2


In [29]:
delivery_df['runs_left'] = round(delivery_df['total_run_x']-delivery_df['current_score'])

delivery_df[['total_run_x', 'current_score', 'runs_left']].head()

,total_run_x,current_score,runs_left
0,131,0,131
1,131,1,130
2,131,2,129
3,131,2,129
4,131,2,129


In [31]:
delivery_df['ballnumber_left'] = 126-(delivery_df['overs']*6+delivery_df['ballnumber'])

delivery_df[['overs', 'ballnumber', 'ballnumber_left']].head(10)

,overs,ballnumber,ballnumber_left
0,0,1,125
1,0,2,124
2,0,3,123
3,0,4,122
4,0,5,121
5,0,6,120
6,1,1,119
7,1,2,118
8,1,3,117
9,1,4,116


In [32]:
delivery_df['player_out']

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
            ...   
144103         NaN
144104         NaN
144105         NaN
144106         NaN
144107    SB Joshi
Name: player_out, Length: 144108, dtype: object

In [33]:
delivery_df['player_out'] = delivery_df['player_out'].fillna("0")

# now we will convert this player_dismissed col into a boolean col
# if the player is not dismissed then it's 0 else it's 1

delivery_df['player_out'] = delivery_df['player_out'].apply(lambda x:x
                                                                      if x=="0" else "1")

# converting string to int

delivery_df['player_out'] = delivery_df['player_out'].astype('int')


delivery_df['player_out'].unique()

array([0, 1])

In [34]:
wickets = delivery_df.groupby('ID').cumsum()['player_out'].values

delivery_df['wickets_left'] = 10-wickets

<ipython-input-34-5adf5e7f9c0a>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.cumsum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  wickets = delivery_df.groupby('ID').cumsum()['player_out'].values


In [35]:
delivery_df['cur_run_rate'] = round((delivery_df['current_score']*6)/(delivery_df['ballnumber_left'] - 120))

# required Run-Rate

delivery_df['req_run_rate'] = round((delivery_df['runs_left']*6)/(delivery_df['ballnumber_left']))

#Current Run-Rate
delivery_df[['cur_run_rate', 'req_run_rate']].head(5)

,cur_run_rate,req_run_rate
0,0.0,6.0
1,2.0,6.0
2,4.0,6.0
3,6.0,6.0
4,12.0,6.0


In [36]:
def resultofmatch(row):

    return 1 if row['BattingTeam'] == row['WinningTeam'] else 0

delivery_df['result'] = delivery_df.apply(resultofmatch,axis=1)

In [ ]:
sns.countplot(delivery_df['result'])

<Axes: ylabel='count'>

In [ ]:
final_df = delivery_df[['BattingTeam','City','runs_left',
                        'ballnumber_left','total_run_x','wickets_left',
                        'req_run_rate','result']]

                        ##we are taking only important columns

final_df.head()

In [ ]:
final_df.shape

In [ ]:
final_df.sample()

In [ ]:
#final_df.isnull().sum()

In [ ]:
final_df.dropna(inplace=True)

In [ ]:
data = final_df.copy()

test = data['result']

train = data.drop(['result'],axis = 1)

train.head()

In [ ]:
final_df.dropna(inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train,test,test_size=0.2,random_state=1)

X_train.shape,X_test.shape

In [ ]:
X_train

In [ ]:
cf = ColumnTransformer(transformers = [
    ('tnf1',OneHotEncoder(sparse=False,drop='first'),['BattingTeam','City'])
],remainder='passthrough')

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

In [ ]:
regressor = DecisionTreeRegressor(random_state=0)

In [ ]:
pipe = Pipeline(steps=[
    ('step1', cf),
    ('step2',LogisticRegression(solver='liblinear'))
])



pipe.fit(X_train,y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tnf1',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['BattingTeam', 'City'])])),
                ('step2', LogisticRegression(solver='liblinear'))])

In [ ]:

import sklearn.metrics as metrics
y_pred = pipe.predict(X_test)
print(metrics.accuracy_score(y_test,y_pred))

0.5780290182777463


In [ ]:
import math
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import *

In [ ]:
def classification_performance(conf_matrix):
    accureacy=(conf_matrix[0][0]+conf_matrix[1][1])/(conf_matrix[0][0]+conf_matrix[0][1]+conf_matrix[1][0]+conf_matrix[1][1])
    recall=(conf_matrix[0][0])/(conf_matrix[0][0]+conf_matrix[1][0])
    specificity=(conf_matrix[1][1])/(conf_matrix[0][1]+conf_matrix[1][1])
    precision=(conf_matrix[0][0])/(conf_matrix[0][0]+conf_matrix[0][1])
    F1_score=(2*precision*recall)/(precision+recall)
    GM=math.sqrt(recall*specificity)
    FPR=1-specificity
    return accureacy,recall,specificity,precision,F1_score,GM,FPR

In [ ]:
pipe.predict_proba(X_test)[10]

array([0.49195606, 0.50804394])

In [ ]:
pipe2 = Pipeline(steps=[
    ('step1',cf),
    ('step2',RandomForestClassifier())
])

pipe2.fit(X_train,y_train)
print(metrics.accuracy_score(y_test,pipe.predict(X_test)))

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.5780290182777463


In [ ]:

from sklearn import tree
pipe2 = Pipeline(steps=[
    ('step1',cf),
    ('step2',tree.DecisionTreeClassifier())
])

pipe2.fit(X_train,y_train)
print(metrics.accuracy_score(y_test,pipe.predict(X_test)))

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.5780290182777463


In [ ]:
pipe2.predict_proba(X_test)[10]

array([0., 1.])

In [ ]:
prediction = pipe2.predict(X_test)
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
confusion_matrix(y_test, prediction)
accuracy_score(y_test, prediction)

0.9928773318258903

In [ ]:
confusion_matrix(y_test, prediction)

array([[13437,    98],
       [   91, 12909]])

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.58      0.63      0.60     13535
           1       0.58      0.52      0.55     13000

    accuracy                           0.58     26535
   macro avg       0.58      0.58      0.58     26535
weighted avg       0.58      0.58      0.58     26535

